# Préparation des données

Montage google drive et décompression des données de bdappv.zip

In [15]:
import sys
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
#sys.path.insert(0,'/content/drive/My Drive/statapps-main/src/')
!unzip /content/drive/MyDrive/bdappv/bdappv.zip > /dev/null

Mounted at /content/drive/
replace __MACOSX/._bdappv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
[A]ll


Importation du fichier dataloader pour pouvoir utiliser ses modules

In [ ]:
!cp /content/drive/MyDrive/statapps-main/src/dataloader.py /content

In [24]:
import dataloader as dtld
import torchvision.models as models

In [28]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from PIL import Image
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [25]:
label_attribution = dtld.LabelAttribution(path_image_google='/content/bdappv/google/img', 
                                   path_mask_google='/content/bdappv/google/mask',
                                   path_metadata='/content/bdappv/metadata.csv',
                                   colonne_identifiant='identifiant',
                                   path_export_train_test='/content/drive/MyDrive',
                                   path_image_ign='/content/bdappv/ign/img',
                                   path_mask_ign='/content/bdappv/ign/mask',
                                   use_img_google=True,
                                   use_img_ign=False
                                    )

In [26]:
label_attribution.run()

In [31]:
path_train='/content/drive/MyDrive/train_data.csv'
path_test='/content/drive/MyDrive/test_data.csv'

transformed_train_dataset  = dtld.CustomImageDataset(path_train,'/content/bdappv/google/img', transform=transforms.Compose([
                                               transforms.Resize(224),
                                               transforms.ToTensor()
                                           ]))
transformed_test_dataset = dtld.CustomImageDataset(path_test,"/content/bdappv/google/img",
                                                transform=transforms.Compose([
                                               transforms.Resize(224),
                                               transforms.ToTensor(),
                                           ]))

In [32]:
print("Nombre d'images dans le train: {}".format(transformed_train_dataset.__len__()))
print("Nombre d'images dans le test: {}".format(transformed_test_dataset.__len__()))

Nombre d'images dans le train: 23045
Nombre d'images dans le test: 5762


In [34]:

train_dataloader = DataLoader(transformed_train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(transformed_test_dataset, batch_size=64, shuffle=True)

 # Implémentation ResNet18

In [35]:
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add
from keras.models import Sequential
from keras.models import Model
import tensorflow as tf

In [ ]:
class ResnetBlock(Model):
    """
    A standard resnet block.
    """

    def __init__(self, channels: int, down_sample=False):
        """
        channels: same as number of convolution kernels
        """
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3, 3)
        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras), as suggested in [2]
        INIT_SCHEME = "he_normal"

        self.conv_1 = Conv2D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_1 = BatchNormalization()
        self.conv_2 = Conv2D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_2 = BatchNormalization()
        self.merge = Add()

        if self.__down_sample:
            # perform down sampling using stride of 2, according to [1].
            self.res_conv = Conv2D(
                self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same")
            self.res_bn = BatchNormalization()

    def call(self, inputs):
        res = inputs

        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = tf.nn.relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = tf.nn.relu(x)
        return out

class ResNet18(Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = Conv2D(64, (7, 7), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = BatchNormalization()
        self.pool_2 = MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(64)
        self.res_1_2 = ResnetBlock(64)
        self.res_2_1 = ResnetBlock(128, down_sample=True)
        self.res_2_2 = ResnetBlock(128)
        self.res_3_1 = ResnetBlock(256, down_sample=True)
        self.res_3_2 = ResnetBlock(256)
        self.res_4_1 = ResnetBlock(512, down_sample=True)
        self.res_4_2 = ResnetBlock(512)
        self.avg_pool = GlobalAveragePooling2D()
        self.flat = Flatten()
        self.fc = Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
            out = res_block(out)
        out = self.avg_pool(out)
        out = self.flat(out)
        out = self.fc(out)
        return out

In [ ]:
model = ResNet18(12)
model.build(input_shape = (None,img_shape,img_shape,3))
#use categorical_crossentropy since the label is one-hot encoded
from keras.optimizers import SGD
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
model.compile(optimizer = "adam",loss='categorical_crossentropy', metrics=["accuracy"]) 
model.summary()